# City of LA Parcel Tax × Zoning Analysis

## Project Overview
This project analyzes property tax efficiency patterns across zoning categories and proximity to transit (based on SB 79) in the City of Los Angeles. By combining spatial data, zoning information, and property tax data, we examine how different land use designations contribute to municipal revenue and model potential impacts of transit-oriented rezoning scenarios. The analysis provides data-driven insights to support policy decisions related to land use optimization near major transit stops.

## Data Sources
* City of LA parcel geometries (`Parcels.gdb`)
* Transit stop proximity tiers based on SB79 (`Metro_Stops_SB_79.geojson`)
* Simplified zoning category mappings (`Parcel tax by zoning - assigned zoning.csv`)
* Property tax data for all LA parcels (`Parcel_Data_2021_Table_-2691831558175163259.csv`)
* City zoning designations (`ZONING_PLY_20250403.geojson`)

## Methodology
* Processing and integrating parcel geometries with tax assessment data
* Categorizing parcels by simplified zoning designations
* Calculating tax revenue per acre across different zoning types
* Analyzing parcels by transit proximity tiers per SB79 definitions
* Modeling financial impacts of potential rezoning scenarios near transit
* Building a regression model training pipeline to attempt to find accurate ways to predict SB 79 up-zoning property tax increases

## Key Inputs
* Parcel-level property tax assessments (2024 snapshot)
* Parcel acreage and geometry
* Zoning designations and simplified category mappings
* Transit stop locations and proximity tiers
* Current land use patterns

## Outputs
* Tax efficiency metrics by zoning category
* Scenario models for potential rezoning impacts
* Visualizations of geographic patterns in tax productivity

## Notes
This analysis focuses on the City of Los Angeles only and uses a zoning dataset that intentionally excludes recent DTLA rezoning changes for consistency purposes. The transit proximity tiers are defined according to SB79 parameters. This analysis is the input to a full [report](https://data.streetsforall.org/blog/sb79_zoning_budget/) on the data.streetsforall.org blog, and should be read alongside that written piece for full context and nuance. 

In [2]:
import math
import re

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rtree
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from shapely import Point
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# Scikit-learn modules for models and evaluation metrics
from sklearn.linear_model import ElasticNet, Lasso, LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR
from tqdm import tqdm


## Property Tax, Zoning, and Parcels Data Collection and Cleaning

In [ ]:
# Read in both CSV files
file1_path = "data/Parcel_Data_2021_Table_-2691831558175163259.csv"

# Read the CSV files
merged_df = pd.read_csv(file1_path)

# Display basic info about the dataframes
print(f"Dataset 1 shape: {merged_df.shape}")


In [ ]:
merged_df = merged_df[merged_df["City Tax Rate Area"] == "LOS ANGELES"]
merged_df

In [32]:
selected = merged_df[
    [
        "Zip Code",
        "City Tax Rate Area",
        "AIN",
        "Roll Year",
        "Tax Rate Area Code",
        "Property Location",
        "Property Use Type",
        "Property Use Code",
        "Number of Buildings",
        "Year Built",
        "Effective Year",
        "Square Footage",
        "Number of Bedrooms",
        "Number of Bathrooms",
        "Number of Units",
        "Land Value",
        "Land Base Year",
        "Improvement Value",
        "Improvement Base Year",
        "Total Value, Land + Improvement",
        "Home Owners Exemption",
        "Real Estate Exemption",
        "Fixture Value",
        "Fixture Exemption",
        "Personal Property Value",
        "Personal Property Exemption",
        "Property taxable?",
        "Total Value",
        "Total Exemption",
        "Taxable Value",
        "Classification",
        "Region Number",
        "Location Latitude",
        "Location Longitude",
    ]
]

### Process and Geocode Zoning Data
Extract relevant information from the zoning designation and map to parcels

In [ ]:
zoning = gpd.read_file("data/ZONING_PLY_20250403.geojson")
zoning = zoning[["zone_cmplt", "geometry"]]
zoning

In [35]:
# Step 1: Convert the regular dataframe to a GeoDataFrame by creating Point geometries
# First, make a copy to avoid modifying the original
selected_gdf = selected.copy()

# Create a geometry column with Point objects from latitude and longitude
selected_gdf["geometry"] = selected_gdf.apply(
    lambda row: Point(row["Location Longitude"], row["Location Latitude"]), axis=1
)

# Convert to a GeoDataFrame
selected_gdf = gpd.GeoDataFrame(selected_gdf, geometry="geometry")

# Make sure both GeoDataFrames have the same CRS (Coordinate Reference System)
# If you know the CRS of your data, set it explicitly
# For example, if your coordinates are in WGS84:
selected_gdf.crs = "EPSG:4326"

# Ensure zoning has the same CRS, or reproject if needed
if zoning.crs != selected_gdf.crs:
    zoning = zoning.to_crs(selected_gdf.crs)

# Step 2: Perform spatial join - this will add zoning attributes to each point
joined_data = gpd.sjoin(selected_gdf, zoning, how="left", predicate="within")

# Step 3: If you want a regular dataframe with the original columns plus zoning info
# (You might want to drop the extra geometry column and index_right column)
result = pd.DataFrame(joined_data.drop(columns=["geometry", "index_right"]))

# Alternatively, if you want to add specific columns from zoning to your original dataframe:
# selected['zoning_type'] = joined_data['zoning_type']  # Replace with your actual column names

In [ ]:
result["ZONE_PREFIX"] = result["zone_cmplt"].str.split("-").str[0]
result

In [38]:
parcels = gpd.read_file("data/Parcels.gdb", columns=["AIN", "geometry"])

In [39]:
parcels["AIN"] = pd.to_numeric(parcels["AIN"], errors="coerce").astype(
    "Int64"
)  # Keeps NaNs


In [40]:
merged_results = pd.merge(
    result,
    parcels,
    on="AIN",  # Join on the AIN column that exists in both dataframes
    how="left",  # Keep all rows from 'result' even if no matching AIN in parcels
)

# If you want to convert the merged result back to a GeoDataFrame
# (in case you need to do more spatial operations later)
# Note: This will use the geometry from the parcels dataframe
merged_gdf = gpd.GeoDataFrame(merged_results, geometry="geometry")

#### Calculate Acreage

In [41]:
projected_gdf = merged_gdf.to_crs("EPSG:2229")

# Calculate area in square feet first (State Plane uses US Survey Feet)
projected_gdf["area_sq_feet"] = projected_gdf.geometry.area

# Convert to acres (1 acre = 43,560 square feet)
projected_gdf["acreage"] = projected_gdf["area_sq_feet"] / 43560

# Drop the intermediate area calculation if you don't need it
projected_gdf = projected_gdf.drop(columns=["area_sq_feet"])

# If you want to round the acreage to a specific number of decimal places
projected_gdf["acreage"] = projected_gdf["acreage"].round(3)

# Convert back to original CRS if needed for further spatial operations
final_gdf = projected_gdf.to_crs(merged_gdf.crs)

In [43]:
# Based on Prop 13 the property tax rate is 1%

final_gdf["Property_Tax_Value"] = final_gdf["Taxable Value"] * 0.01

In [44]:
final_gdf = final_gdf.dropna(subset=["acreage"])

In [ ]:
final_gdf["Property_Tax_Value"].sum() * 0.2478


In [47]:
# Function to extract first letter of zoning code
def extract_first_letter(zone_code):
    if pd.isna(zone_code):
        return None

    # Remove any brackets or parentheses and their contents
    # This regex looks for patterns like [...], (...), etc.
    cleaned_code = re.sub(r"[\[\(].*?[\]\)]", "", zone_code)
    cleaned_prefix = cleaned_code

    # Remove any remaining special characters
    cleaned_code = re.sub(r"[^a-zA-Z0-9]", "", cleaned_code)
    cleaned_code = cleaned_code.replace("Q", "")

    # Take the first letter if there is one
    if cleaned_code and len(cleaned_code) > 0:
        return cleaned_code[0], cleaned_prefix
    else:
        return None, None


def extract_zone_district_clean(zone_code):
    if pd.isna(zone_code):
        return None

    # Remove anything inside () or [] and the brackets themselves
    cleaned = re.sub(r"[\(\[].*?[\)\]]", "", zone_code).strip()

    cleaned = cleaned.replace("Q", "")

    # Find the first occurrence of a dash and keep one character after it
    match = re.match(r"([A-Za-z0-9]+-\w?)", cleaned)
    if match:
        return match.group(1)

    return None


# Apply to final_gdf
final_gdf["zone_district_clean"] = final_gdf["zone_cmplt"].apply(
    extract_zone_district_clean
)


# Apply and split into two columns properly
zone_extracted = final_gdf["ZONE_PREFIX"].apply(extract_first_letter).apply(pd.Series)
zone_extracted.columns = ["zone_type", "zone_prefix_clean"]

# Add them to the original DataFrame
final_gdf[["zone_type", "zone_prefix_clean"]] = zone_extracted

# Remove all 'Q' characters from zone_prefix_clean and zone_district_clean
final_gdf["zone_prefix_clean"] = final_gdf["zone_prefix_clean"].str.replace(
    "Q", "", regex=False
)
final_gdf["zone_district_clean"] = final_gdf["zone_district_clean"].str.replace(
    "Q", "", regex=False
)


In [ ]:
# Define logic for zone_consolidation
def consolidate_zone(row):
    if row["zone_prefix_clean"] in ["C2", "C4"]:
        return row["zone_district_clean"]
    return row["zone_prefix_clean"]


# Apply the function row-wise
final_gdf["zone_consolidation"] = final_gdf.apply(consolidate_zone, axis=1)
print(final_gdf["zone_consolidation"].value_counts())

In [ ]:
print(final_gdf["ZONE_PREFIX"].unique())

In [ ]:
print(final_gdf["zone_prefix_clean"].unique())

In [ ]:
final_gdf[final_gdf["zone_consolidation"] == None]

In [ ]:
print(final_gdf["zone_consolidation"].unique())

#### Import SFA's zoning buckets and map to parcels

In [ ]:
# Load the spreadsheet (update path as needed)
zoning_map_df = pd.read_csv("data/Parcel tax by zoning - assigned zoning.csv")

# Create the dictionary mapping
abridged_to_bucket = dict(
    zip(zoning_map_df["Abridged Zoning Class"], zoning_map_df["Assigned Zoning Bucket"])
)

print(abridged_to_bucket)

In [ ]:
# Apply the dictionary to create a new column
final_gdf["assigned_zoning_bucket"] = final_gdf["zone_consolidation"].map(
    abridged_to_bucket
)
final_gdf

In [ ]:
final_gdf["assigned_zoning_bucket"].value_counts()

In [ ]:
missing_count = final_gdf["assigned_zoning_bucket"].isna().sum()
print(f"Rows with missing zoning bucket: {missing_count}")


In [ ]:
final_buckets = final_gdf[final_gdf["assigned_zoning_bucket"] != "Ignore"]
final_buckets = final_buckets.dropna(subset=["assigned_zoning_bucket"])
final_buckets

In [ ]:
final_geom = final_buckets[
    [
        "geometry",
        "AIN",
        "Property Location",
        "zone_type",
        "acreage",
        "Number of Units",
        "Property_Tax_Value",
        "zone_cmplt",
        "zone_consolidation",
        "assigned_zoning_bucket",
        "Location Latitude",
        "Location Longitude",
        "Taxable Value",
        "Property taxable?",
    ]
]
final_geom.info()

#### Inspect data for issues before beggining merge process

In [ ]:
# Basic statistics on your tax column
print("Tax value summary statistics:")
print(final_geom["Property_Tax_Value"].describe())

# Check the largest values - they might be outliers
print("\nTop 10 largest tax values:")
print(
    final_geom.nlargest(10, "Property_Tax_Value")[
        ["AIN", "assigned_zoning_bucket", "Property_Tax_Value", "acreage"]
    ]
)

# Are there any negative or zero values?
print(
    f"\nCount of zero/negative tax values: {len(final_geom[final_geom['Property_Tax_Value'] <= 0])}"
)

# Check spatial distribution - are all properties truly in LA City?
if "geometry" in final_gdf.columns:
    # Calculate area of study
    bounds = final_geom.total_bounds
    area_km2 = (
        (bounds[2] - bounds[0]) * (bounds[3] - bounds[1]) / 1000000
    )  # rough estimate
    print(f"\nApproximate geographic area covered: {area_km2:.2f} km²")

    # LA City is roughly 1,300 km² - if your area is much larger, you have data outside the city

# Calculate mean tax per property
mean_tax = final_geom["Property_Tax_Value"].mean()
count = len(final_geom)
print(f"\nMean tax per property: ${mean_tax:.2f}")
print(f"Number of properties: {count}")

In [ ]:
final_geom["acreage"].sum()

In [64]:
# final_geom.to_file("final_geom.gpkg", driver="GPKG")

## Merge Parcel Geospatial Data
Properties like condos and gated communities feature duplicates geometry, throwing off acreage calculation so it is neccesary to merge overlapping parcels together

In [65]:
# Define chunk processing function at module level so it can be pickled
def _process_chunk(chunk_data):
    """
    Process a chunk of polygons to find overlaps.

    Parameters:
    chunk_data: Tuple containing (chunk_idx, gdf, areas, chunk_size, n, overlap_threshold, debug)

    Returns:
    Tuple of (rows, cols) with indices of overlapping polygons
    """
    chunk_idx, gdf, areas, chunk_size, n, overlap_threshold, debug = chunk_data

    start_idx = chunk_idx * chunk_size
    end_idx = min(start_idx + chunk_size, n)
    chunk_indices = range(start_idx, end_idx)

    # Create R-tree spatial index for this chunk
    idx = rtree.index.Index()
    for i, idx_val in enumerate(chunk_indices):
        # Skip zero area geometries when building index
        if areas[idx_val] <= 0:
            continue
        idx.insert(i, gdf.geometry.iloc[idx_val].bounds)

    # Find overlaps
    rows = []
    cols = []

    for i, idx1 in enumerate(chunk_indices):
        # Skip zero area geometries
        if areas[idx1] <= 0:
            continue

        geom1 = gdf.geometry.iloc[idx1]
        area1 = areas[idx1]
        bounds = geom1.bounds

        # Find potential overlaps using spatial index
        for j in idx.intersection(bounds):
            idx2 = start_idx + j

            # Only check forward to avoid duplicates
            if idx2 <= idx1:
                continue

            # Skip zero area geometries
            if areas[idx2] <= 0:
                continue

            # Get geometry
            geom2 = gdf.geometry.iloc[idx2]

            # Quick intersection check
            if not geom1.intersects(geom2):
                continue

            try:
                # Calculate intersection
                intersection = geom1.intersection(geom2)
                intersection_area = intersection.area

                # Calculate overlap ratios
                ratio1 = intersection_area / area1
                ratio2 = intersection_area / areas[idx2]

                # Check if either ratio exceeds threshold
                if ratio1 >= overlap_threshold or ratio2 >= overlap_threshold:
                    rows.append(idx1)
                    cols.append(idx2)
            except Exception as e:
                if debug:
                    print(f"Error checking overlap between {idx1} and {idx2}: {str(e)}")

    return rows, cols


# Define component processing function at module level
def _process_component(component_data):
    """
    Process a component (group of overlapping polygons).
    Instead of merging geometries, use geometry with highest acreage.

    Parameters:
    component_data: Tuple containing (component_id, gdf, labels)

    Returns:
    Tuple of (merged_data_dict, merge_count)
    """
    component_id, gdf, labels = component_data

    # Get indices for this component
    component_indices = np.where(labels == component_id)[0]

    if len(component_indices) == 1:
        # Single property, no merging needed
        return gdf.iloc[component_indices[0]].to_dict(), 0
    else:
        # Get the properties to merge
        group_data = gdf.iloc[component_indices]

        # Find index of property with highest acreage
        if "acreage" in group_data.columns:
            max_acreage_idx = group_data["acreage"].idxmax()
            # Use geometry of property with highest acreage
            main_geom = group_data.loc[max_acreage_idx, "geometry"]
        else:
            # If no acreage column, use the largest geometry by area
            max_area_idx = group_data["_area"].idxmax()
            main_geom = group_data.loc[max_area_idx, "geometry"]

        # Prepare aggregated data
        agg_data = {}

        # For all numeric columns, sum them
        for col in group_data.select_dtypes(include=np.number).columns:
            if col not in ["Location Latitude", "Location Longitude", "_area"]:
                agg_data[col] = group_data[col].sum()

        # Keep the max acreage value
        if "acreage" in group_data.columns:
            agg_data["acreage"] = group_data["acreage"].max()

        # For lat/long, take median values
        if "Location Latitude" in group_data.columns:
            agg_data["Location Latitude"] = group_data["Location Latitude"].median()
        if "Location Longitude" in group_data.columns:
            agg_data["Location Longitude"] = group_data["Location Longitude"].median()

        # For string columns, join unique values
        for col in group_data.select_dtypes(include=["object"]).columns:
            unique_values = group_data[col].dropna().astype(str).unique()
            agg_data[col] = ", ".join(unique_values) if len(unique_values) > 0 else None

        # For boolean columns, use logical OR
        for col in group_data.select_dtypes(include=["bool"]).columns:
            agg_data[col] = any(group_data[col])

        # Set the geometry to the one with highest acreage
        agg_data["geometry"] = main_geom

        return agg_data, len(component_indices)


def merge_overlapping_properties_serial(
    gdf, overlap_threshold=0.8, chunk_size=10000, debug=True
):
    """
    Memory-efficient version that processes chunks sequentially.
    Uses geometry with highest acreage for each merged group.

    Parameters:
    gdf (GeoDataFrame): Input GeoDataFrame with property polygons
    overlap_threshold (float): Minimum overlap ratio required for merging (0.0 to 1.0)
    chunk_size (int): Number of polygons to process in each chunk
    debug (bool): Whether to print debug messages

    Returns:
    GeoDataFrame: Processed GeoDataFrame with merged properties
    """
    if debug:
        print("Starting memory-efficient property polygon merging...")

    try:
        # Make a copy to avoid modifying the original
        gdf = gdf.copy()
        original_crs = gdf.crs

        # Convert to UTM Zone 11N for accurate spatial calculations
        projected_crs = "EPSG:32611"  # UTM Zone 11N (meters)
        if debug:
            print(f"Converting from {original_crs} to {projected_crs} for processing")

        # Convert to projected CRS
        gdf = gdf.to_crs(projected_crs)

        # Fix any invalid geometries
        if debug:
            print("Fixing invalid geometries...")
        gdf["geometry"] = gdf.geometry.apply(
            lambda geom: geom.buffer(0) if not geom.is_valid else geom
        )

        # Pre-compute areas for all polygons
        if debug:
            print("Pre-computing areas...")
        gdf["_area"] = gdf.geometry.area
        areas = gdf["_area"].values

        if debug:
            print(
                f"Area statistics: Min={min(areas):.2f}, Max={max(areas):.2f}, Mean={np.mean(areas):.2f}"
            )

        # Calculate number of chunks
        n = len(gdf)
        n_chunks = math.ceil(n / chunk_size)
        if debug:
            print(f"Processing {n} properties in {n_chunks} chunks of {chunk_size}...")

        # Process all chunks serially to save memory
        all_rows = []
        all_cols = []

        for chunk_idx in tqdm(range(n_chunks), disable=not debug):
            rows, cols = _process_chunk(
                (chunk_idx, gdf, areas, chunk_size, n, overlap_threshold, debug)
            )
            all_rows.extend(rows)
            all_cols.extend(cols)

            # Free memory periodically
            if chunk_idx % 10 == 0 and debug:
                import gc

                gc.collect()

        # Check if any overlaps were found
        if not all_rows:
            if debug:
                print("No overlapping properties detected with the current threshold.")
                print(
                    f"Try a lower overlap_threshold value (current: {overlap_threshold})"
                )
            return gdf.drop(columns=["_area"]).to_crs(original_crs)

        # Create sparse matrix for the entire dataset
        if debug:
            print("Creating adjacency matrix...")

        # Add symmetric relationships (more efficiently)
        all_rows.extend(all_cols)
        all_cols.extend(all_rows[: -len(all_cols)])
        data = np.ones(len(all_rows), dtype=np.int8)

        adjacency_matrix = csr_matrix((data, (all_rows, all_cols)), shape=(n, n))

        # Find connected components (groups of overlapping properties)
        if debug:
            print("Finding connected components...")
        n_components, labels = connected_components(adjacency_matrix, directed=False)

        if debug:
            print(f"Found {n_components} distinct property groups")

        # Check if any merging actually happened
        if n_components == n:
            if debug:
                print("No properties were merged.")
            return gdf.drop(columns=["_area"]).to_crs(original_crs)

        # Process components sequentially to save memory
        new_rows = []
        merged_count = 0

        if debug:
            print("Processing and merging components...")

        # Pre-allocate a list to store processed components
        processed_components = []

        for component_id in tqdm(range(n_components), disable=not debug):
            row_dict, count = _process_component((component_id, gdf, labels))
            processed_components.append(row_dict)
            merged_count += count

            # Free memory periodically
            if component_id % 1000 == 0 and debug:
                import gc

                gc.collect()

        # Create GeoDataFrame in one operation from processed components
        if debug:
            print("Creating final GeoDataFrame...")
        merged_gdf = gpd.GeoDataFrame(processed_components, crs=projected_crs)

        if debug:
            print(
                f"Original properties: {len(gdf)}, Merged properties: {len(merged_gdf)}"
            )
            print(f"Properties merged into groups: {merged_count}")

            if "Property_Tax_Value" in merged_gdf.columns:
                print(
                    f"Total tax before merging: ${gdf['Property_Tax_Value'].sum():,.2f}"
                )
                print(
                    f"Total tax after merging: ${merged_gdf['Property_Tax_Value'].sum():,.2f}"
                )

        # Convert back to original CRS
        merged_gdf = merged_gdf.to_crs(original_crs)

        # Drop internal calculation columns
        if "_area" in merged_gdf.columns:
            merged_gdf = merged_gdf.drop(columns=["_area"])

        return merged_gdf

    except Exception as e:
        if debug:
            print(f"Error during merging: {str(e)}")
            import traceback

            traceback.print_exc()
        return gdf.drop(columns=["_area"] if "_area" in gdf.columns else []).to_crs(
            original_crs
        )

In [ ]:
result = merge_overlapping_properties_serial(
    final_geom, overlap_threshold=0.8, chunk_size=10000
)

In [ ]:
result.info()

In [ ]:
result["assigned_zoning_bucket"].unique()

#### Performing some cleanup of the merge process, as there are a few parcels merged across zones

In [ ]:
result[result["assigned_zoning_bucket"].isin(["R3, R2"])]

In [ ]:
result[result["assigned_zoning_bucket"].isin(["R1, R2"])]

In [ ]:
result[result["assigned_zoning_bucket"].isin(["R5, R4"])]

In [10]:
result.loc[
    result["assigned_zoning_bucket"].isin(["R3, R2"]), "assigned_zoning_bucket"
] = "R3"

In [11]:
result.loc[
    result["assigned_zoning_bucket"].isin(["R1, R2"]), "assigned_zoning_bucket"
] = "R2"

In [12]:
result.loc[
    result["assigned_zoning_bucket"].isin(["R5, R4"]), "assigned_zoning_bucket"
] = "R5"

In [13]:
result.loc[
    result["assigned_zoning_bucket"].isin(["R4, R2"]), "assigned_zoning_bucket"
] = "R4"


In [14]:
result.loc[
    result["assigned_zoning_bucket"].isin(["R4, R3"]), "assigned_zoning_bucket"
] = "R4"


In [ ]:
result["assigned_zoning_bucket"].value_counts()

In [ ]:
# Create a mapping of AIN to Effective Year from final_gdf
ain_to_effective_year = final_gdf.set_index("AIN")["Effective Year"].to_dict()

# Map this to the result DataFrame based on the AIN column
result["Effective Year"] = result["AIN"].map(ain_to_effective_year)


In [ ]:
invalid_count = result["Effective Year"].isna().sum()
print(f"Number of rows without a valid Effective Year: {invalid_count}")


## Calculate Zoning Tax/Acre Efficiency
Calculate for both full dataset and post-2014 Effective Year parcels - see report for explanation

In [ ]:
# Group by ZONE_PREFIX and calculate aggregates
zone_summary = {}

# Get unique zone prefixes
zone_prefixes = result["assigned_zoning_bucket"].unique()
print(zone_prefixes)
for zone in zone_prefixes:
    # Filter for just this zone prefix
    zone_data = result[result["assigned_zoning_bucket"] == zone]

    # Skip if zone is None/NaN
    if pd.isna(zone):
        continue

    try:
        # Calculate metrics with error handling
        total_acreage = zone_data["acreage"].sum()
        total_property_tax = zone_data["Property_Tax_Value"].sum()

        # Filter for recent Effective Year (after 2015)
        recent_data = zone_data[zone_data["Effective Year"] > 2015]
        total_acreage_recent = recent_data["acreage"].sum()
        total_property_tax_recent = recent_data["Property_Tax_Value"].sum()
    except KeyError as e:
        print(f"Missing key {e} in DataFrame. Skipping zone: {zone}")
        continue  # Skip this zone if keys are missing

    # Avoid division by zero
    tax_per_acre = total_property_tax / total_acreage if total_acreage > 0 else 0
    tax_per_acre_recent = (
        total_property_tax_recent / total_acreage_recent
        if total_acreage_recent > 0
        else 0
    )

    # Store in dictionary
    zone_summary[zone] = {
        "total_acreage": round(total_acreage, 2),
        "total_property_tax": round(total_property_tax, 2),
        "tax_per_acre": round(tax_per_acre, 2),
        "tax_per_acre_recent": round(tax_per_acre_recent, 2),
    }

# Now zone_summary is a dictionary with all the metrics by zone prefix
print(zone_summary)


In [ ]:
# Extract tax_per_acre values, filtering out any missing values
tax_per_acre_values = {
    k: v["tax_per_acre"] for k, v in zone_summary.items() if "tax_per_acre" in v
}

# Sort by tax_per_acre in descending order
sorted_data = sorted(tax_per_acre_values.items(), key=lambda x: x[1], reverse=True)

# Display results
for letter, tax in sorted_data:
    print(f"{letter}: {tax}")

In [ ]:
# Extract tax_per_acre values, filtering out any missing values
tax_per_acre_values = {
    k: v["tax_per_acre_recent"]
    for k, v in zone_summary.items()
    if "tax_per_acre_recent" in v
}

# Sort by tax_per_acre in descending order
sorted_data = sorted(tax_per_acre_values.items(), key=lambda x: x[1], reverse=True)

# Display results
for letter, tax in sorted_data:
    print(f"{letter}: {tax}")

#### The following cells allow for exporting the merged parcel dataset and saving it back so that the full process doesn't need to be run multiple times

In [82]:
# result.to_file("parcels_final.gpkg", driver="GPKG")

In [3]:
# result_read = gpd.read_file("parcels_final.gpkg")

In [ ]:
# result = result_read
result

In [5]:
# Function to extract ZIP code safely
def extract_zip(location):
    if not isinstance(location, str):
        return None
    zips = re.findall(r"\b\d{5}\b", location)
    return zips[-1] if zips else None


# Create new column with ZIP code
result["zip_code"] = result["Property Location"].apply(extract_zip)


In [ ]:
len(result["zip_code"].unique())

In [ ]:
## Regress

## Regression Analysis of Parcel Up-Zoning
Training pipeline which builds several types of regression models on the data set using different combinations of features. Goal is to predict property tax values of an upzoned property based on size and location. 

In [201]:
def trim_outliers(df_, lower_q=0.01, upper_q=0.99):
    """
    Remove outliers from the dataframe based on quantile ranges
    """
    if len(df_) < 4:  # Not enough data to trim meaningfully
        return df_

    return df_[
        df_["Property_Tax_Value"].between(
            df_["Property_Tax_Value"].quantile(lower_q),
            df_["Property_Tax_Value"].quantile(upper_q),
        )
        & df_["acreage"].between(
            df_["acreage"].quantile(lower_q),
            df_["acreage"].quantile(upper_q),
        )
    ]


def prepare_data(
    df, min_year=0, trim_outliers_flag=False, lower_q=0.01, upper_q=0.99, use_age=False
):
    """
    Prepare data for regression modeling
    Optionally trim outliers and choose between filtering by year or using parcel age
    """
    if use_age:
        # Use full dataset with parcel age as a feature
        df_processed = df.copy()
    else:
        # Filter by year
        df_processed = df[df["Effective Year"] > min_year].copy()

    print(len(df_processed))

    # Drop rows with missing required fields
    df_processed = df_processed.dropna(
        subset=[
            "acreage",
            "Location Latitude",
            "Location Longitude",
            "Property_Tax_Value",
            "assigned_zoning_bucket",
            "Effective Year",  # Added to ensure Parcel_age is valid
        ]
    )

    print(f"After dropping NaN in required fields: {len(df_processed)}")

    print(len(df_processed))

    # Always calculate parcel age for all records
    df_processed["Parcel_age"] = 2025 - df_processed["Effective Year"]

    # Simple fix: Drop any rows with NaN values in any column that will be used as a feature
    # df_processed = df_processed.dropna()
    df_processed = df_processed.dropna(
        subset=[
            "acreage",
            "Location Latitude",
            "Location Longitude",
            "Property_Tax_Value",
            "assigned_zoning_bucket",
        ]
    )

    print(len(df_processed))

    # Trim outliers if requested
    if trim_outliers_flag:
        original_len = len(df_processed)
        df_processed = trim_outliers(df_processed, lower_q, upper_q)
        trimmed_count = original_len - len(df_processed)
        print(
            f"Trimmed {trimmed_count} outliers ({trimmed_count / original_len:.1%} of data)"
        )

    print(len(df_processed))

    return df_processed


def compare_regression_models(
    df,
    zone_prefixes,
    feature_set="acreage_only",
    min_year=2015,
    trim_outliers_flag=False,
    lower_q=0.01,
    upper_q=0.99,
):
    """
    Compare different regression models using different feature sets

    feature_set options:
    - "acreage_only": just acreage (filtered by year)
    - "acreage_latlon": acreage and lat/lon (filtered by year)
    - "acreage_zipcode": acreage and zip code (filtered by year)
    - "acreage_age_latlon": full dataset with acreage, parcel age, and lat/lon
    - "acreage_age_zipcode": full dataset with acreage, parcel age, and zip code
    """
    # Determine if we're using parcel age (full dataset) or filtering by year
    use_age = "_age" in feature_set
    print("use age", use_age)
    # Prepare data with appropriate features
    df_processed = prepare_data(
        df,
        min_year=min_year if not use_age else 0,
        trim_outliers_flag=trim_outliers_flag,
        lower_q=lower_q,
        upper_q=upper_q,
        use_age=use_age,
    )

    # If using zip codes, add the one-hot encoding
    if "zipcode" in feature_set:
        zip_dummies = pd.get_dummies(df_processed["zip_code"], prefix="zip")
        df_processed = pd.concat([df_processed, zip_dummies], axis=1)

    # Models to evaluate
    models = {
        "Linear Regression": LinearRegression(),
        "Ridge Regression": Ridge(alpha=1.0),
        "Lasso Regression": Lasso(alpha=0.1),
        "ElasticNet": ElasticNet(alpha=0.1, l1_ratio=0.5),
        "SVR": Pipeline(
            [("scaler", StandardScaler()), ("svr", LinearSVR(max_iter=10000))]
        ),
        "Random Forest": RandomForestRegressor(
            n_estimators=50, random_state=42, n_jobs=4
        ),
        "Gradient Boosting": GradientBoostingRegressor(
            n_estimators=50, random_state=42
        ),
    }

    # Skip NaN zone if present
    valid_zones = [zone for zone in zone_prefixes if not pd.isna(zone)]

    # Dictionary to store results
    results = {}

    # Set up feature columns based on feature_set
    base_features = ["acreage"]

    if "age" in feature_set:
        base_features.append("Parcel_age")

    if "latlon" in feature_set:
        base_features.extend(["Location Latitude", "Location Longitude"])

    # For each zone
    for zone in valid_zones:
        zone_data = df_processed[df_processed["assigned_zoning_bucket"] == zone].copy()

        # Skip if insufficient data (need at least 10 samples for meaningful results)
        if len(zone_data) < 10:
            print(
                f"Skipping zone {zone} - insufficient data (only {len(zone_data)} samples)"
            )
            continue

        print(f"\nProcessing zone: {zone} (Data: {len(zone_data)} samples)")

        # Prepare features based on feature_set
        feature_columns = base_features.copy()

        # Add zip code features if needed
        if "zipcode" in feature_set:
            zip_columns = [col for col in zone_data.columns if col.startswith("zip_")]
            feature_columns.extend(zip_columns)

        # Check for NaN values in feature columns
        print("NaN values in feature columns before filtering:")
        for col in feature_columns:
            nan_count = zone_data[col].isna().sum()
            if nan_count > 0:
                print(f"  {col}: {nan_count} NaNs")

        # Filter rows with any NaN in feature columns
        zone_data = zone_data.dropna(subset=feature_columns)
        print(f"After removing NaNs in features: {len(zone_data)} samples")

        # Skip if insufficient data after NaN removal
        if len(zone_data) < 10:
            print(f"Skipping zone {zone} after NaN removal - insufficient data")
            continue

        X = zone_data[feature_columns].values
        y = zone_data["Property_Tax_Value"].values

        # Try to split data for validation - 70% train, 30% test
        try:
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.3, random_state=42
            )

            zone_results = []

            # Evaluate each model
            for name, model in models.items():
                try:
                    # Train model
                    model.fit(X_train, y_train)

                    # Make predictions
                    y_pred = model.predict(X_test)

                    # Calculate metrics
                    r2 = r2_score(y_test, y_pred)
                    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
                    mae = mean_absolute_error(y_test, y_pred)

                    # Cross validation for robustness (ensure we have enough folds)
                    cv_folds = min(
                        5, len(zone_data) // 3
                    )  # Ensure at least 3 samples per fold
                    if cv_folds >= 2:  # Need at least 2 folds
                        cv_scores = cross_val_score(
                            model, X, y, cv=cv_folds, scoring="r2"
                        )
                        cv_r2 = cv_scores.mean()
                    else:
                        cv_r2 = np.nan  # Cannot perform CV with too few samples

                    zone_results.append(
                        {
                            "zone": zone,
                            "model": name,
                            "r2_score": r2,
                            "cv_r2_score": cv_r2,
                            "rmse": rmse,
                            "mae": mae,
                            "samples": len(zone_data),
                            "feature_set": feature_set,
                            "num_features": X.shape[1],
                            "outliers_trimmed": trim_outliers_flag,
                        }
                    )

                    print(
                        f"  {name}: R² = {r2:.4f}, CV R² = {cv_r2:.4f}, RMSE = {rmse:.2f}, MAE = {mae:.2f}"
                    )

                except Exception as e:
                    print(f"  Error with {name}: {str(e)}")

            results[zone] = zone_results

        except ValueError as e:
            print(f"  Error splitting data for zone {zone}: {str(e)}")

    return results


def summarize_results(results, feature_set):
    """
    Summarize regression results and create visualizations
    """
    # Flatten results into a dataframe
    all_results = []
    for zone, models in results.items():
        all_results.extend(models)

    df_results = pd.DataFrame(all_results)

    if df_results.empty:
        print("No results to summarize.")
        return df_results

    # Print best model for each zone
    print(f"\n--- BEST MODEL FOR EACH ZONING BUCKET (Feature Set: {feature_set}) ---")
    for zone in df_results["zone"].unique():
        zone_df = df_results[df_results["zone"] == zone]

        # Find index of best model based on R²
        if not zone_df.empty and not all(pd.isna(zone_df["r2_score"])):
            best_idx = zone_df["r2_score"].idxmax()
            if pd.notna(best_idx):  # Ensure we have a valid index
                best_model = zone_df.loc[best_idx]
                outlier_status = (
                    "with outliers trimmed"
                    if best_model.get("outliers_trimmed", False)
                    else "all data"
                )
                print(
                    f"\nZONE: {zone} (Samples: {best_model['samples']}, Features: {best_model['num_features']}, {outlier_status})"
                )
                print(f"Best model: {best_model['model']}")
                print(f"R² Score: {best_model['r2_score']:.4f}")
                if pd.notna(best_model["cv_r2_score"]):
                    print(f"CV R² Score: {best_model['cv_r2_score']:.4f}")
                else:
                    print("CV R² Score: N/A (insufficient data for cross-validation)")
                print(f"RMSE: {best_model['rmse']:.2f}")
                print(f"MAE: {best_model['mae']:.2f}")

    # Plot R² comparison across zones and models
    for zone in df_results["zone"].unique():
        zone_df = df_results[df_results["zone"] == zone]

        if zone_df.empty:
            continue

        plt.figure(figsize=(12, 6))

        # Sort by R² score
        zone_df = zone_df.sort_values("r2_score", ascending=False)

        plt.bar(zone_df["model"], zone_df["r2_score"], alpha=0.7)
        plt.axhline(
            y=0.7, color="r", linestyle="--", label="R² = 0.7 (Good fit threshold)"
        )

        # Title with feature set and outlier info
        outlier_suffix = (
            " (Outliers Trimmed)"
            if zone_df.iloc[0].get("outliers_trimmed", False)
            else ""
        )
        plt.title(
            f"Model R² Comparison for Zone: {zone} - {feature_set}{outlier_suffix}",
            fontsize=14,
        )

        plt.xlabel("Model")
        plt.ylabel("R² Score")
        plt.xticks(rotation=45, ha="right")
        plt.ylim(0, 1.0)
        plt.legend()
        plt.tight_layout()

        # Save with descriptive filename
        outlier_filename = (
            "_trimmed" if zone_df.iloc[0].get("outliers_trimmed", False) else ""
        )
        # plt.savefig(
        #     f"model_comparison_zone_{zone}_{feature_set}{outlier_filename}.png",
        #     dpi=300,
        #     bbox_inches="tight",
        # )
        plt.close()

    return df_results


def find_best_models_multi_metric(
    best_df, weight_r2=0.3, weight_cv_r2=0.4, weight_rmse=0.3
):
    """
    Find the best model for each zone using a weighted combination of R², CV R², and RMSE.

    Parameters:
    -----------
    best_df : pandas DataFrame
        DataFrame containing model results with columns: 'zone', 'feature_set', 'best_model',
        'r2_score', 'cv_r2_score', 'rmse', and 'samples'
    weight_r2 : float, default=0.3
        Weight for R² score (higher is better)
    weight_cv_r2 : float, default=0.4
        Weight for CV R² score (higher is better)
    weight_rmse : float, default=0.3
        Weight for negative RMSE (lower RMSE is better)

    Returns:
    --------
    dict: Dictionary with zone keys and best model information as values
    """
    # Input validation
    total_weight = weight_r2 + weight_cv_r2 + weight_rmse
    if not np.isclose(total_weight, 1.0):
        print(f"Warning: Weights sum to {total_weight}, not 1.0. Normalizing weights.")
        weight_r2 /= total_weight
        weight_cv_r2 /= total_weight
        weight_rmse /= total_weight

    print("\n--- OVERALL BEST MODEL & FEATURE SET FOR EACH ZONE (MULTI-METRIC) ---")
    print(
        f"Weights: R² = {weight_r2:.2f}, CV R² = {weight_cv_r2:.2f}, RMSE = {weight_rmse:.2f}"
    )

    best_models = {}

    for zone in best_df["zone"].unique():
        zone_best_df = best_df[best_df["zone"] == zone].copy()

        if zone_best_df.empty:
            continue

        # Handle NaN values in metrics
        zone_best_df["cv_r2_score"] = zone_best_df["cv_r2_score"].fillna(
            zone_best_df["r2_score"] * 0.8
        )

        # Normalize metrics to [0,1] range for fair comparison
        # For R² and CV R², higher is better (1 is perfect)
        # For RMSE, lower is better (0 is perfect)

        if len(zone_best_df) > 1:  # Only normalize if more than one model
            # For R² scores: normalize to [0,1] where 1 is best (highest R²)
            r2_min = zone_best_df["r2_score"].min()
            r2_max = zone_best_df["r2_score"].max()
            r2_range = r2_max - r2_min

            cv_r2_min = zone_best_df["cv_r2_score"].min()
            cv_r2_max = zone_best_df["cv_r2_score"].max()
            cv_r2_range = cv_r2_max - cv_r2_min

            # For RMSE: normalize to [0,1] where 1 is best (lowest RMSE)
            rmse_min = zone_best_df["rmse"].min()
            rmse_max = zone_best_df["rmse"].max()
            rmse_range = rmse_max - rmse_min

            # Avoid division by zero
            if r2_range > 0:
                zone_best_df["r2_norm"] = (zone_best_df["r2_score"] - r2_min) / r2_range
            else:
                zone_best_df["r2_norm"] = 1.0  # All values are the same

            if cv_r2_range > 0:
                zone_best_df["cv_r2_norm"] = (
                    zone_best_df["cv_r2_score"] - cv_r2_min
                ) / cv_r2_range
            else:
                zone_best_df["cv_r2_norm"] = 1.0  # All values are the same

            if rmse_range > 0:
                # Invert RMSE so that lower is better (1 - normalized value)
                zone_best_df["rmse_norm"] = 1 - (
                    (zone_best_df["rmse"] - rmse_min) / rmse_range
                )
            else:
                zone_best_df["rmse_norm"] = 1.0  # All values are the same
        else:
            # If only one model, set normalized values to 1
            zone_best_df["r2_norm"] = 1.0
            zone_best_df["cv_r2_norm"] = 1.0
            zone_best_df["rmse_norm"] = 1.0

        # Calculate weighted composite score
        zone_best_df["composite_score"] = (
            weight_r2 * zone_best_df["r2_norm"]
            + weight_cv_r2 * zone_best_df["cv_r2_norm"]
            + weight_rmse * zone_best_df["rmse_norm"]
        )

        # Find the model with the highest composite score
        overall_best_idx = zone_best_df["composite_score"].idxmax()
        overall_best = zone_best_df.loc[overall_best_idx]

        # Print detailed results
        print(f"\nZONE: {zone} (Samples: {overall_best['samples']})")
        print(f"Best feature set: {overall_best['feature_set']}")
        print(f"Best model: {overall_best['best_model']}")
        print(
            f"R² Score: {overall_best['r2_score']:.4f} (normalized: {overall_best['r2_norm']:.4f})"
        )
        print(
            f"CV R² Score: {overall_best['cv_r2_score']:.4f} (normalized: {overall_best['cv_r2_norm']:.4f})"
        )
        print(
            f"RMSE: {overall_best['rmse']:.2f} (normalized: {overall_best['rmse_norm']:.4f})"
        )
        print(f"Composite Score: {overall_best['composite_score']:.4f}")

        # Add to results dictionary
        best_models[zone] = overall_best.to_dict()

        # Top 3 models by composite score for this zone
        print("\nTop models by composite score:")
        top_models = zone_best_df.sort_values("composite_score", ascending=False).head(
            3
        )

        for i, (_, model) in enumerate(top_models.iterrows(), 1):
            print(
                f"{i}. {model['best_model']} ({model['feature_set']}): "
                f"Score = {model['composite_score']:.4f}, "
                f"R² = {model['r2_score']:.4f}, "
                f"CV R² = {model['cv_r2_score']:.4f}, "
                f"RMSE = {model['rmse']:.2f}"
            )

    return best_models


def visualize_metric_comparison(best_df, zone, metrics=None):
    """
    Create a parallel coordinates plot to visualize multiple metrics across models
    for a specific zone.

    Parameters:
    -----------
    best_df : pandas DataFrame
        DataFrame containing model results
    zone : str
        The zone to visualize
    metrics : list, optional
        List of metrics to include. Defaults to ['r2_score', 'cv_r2_score', 'rmse']
    """
    import matplotlib.pyplot as plt
    from pandas.plotting import parallel_coordinates

    if metrics is None:
        metrics = ["r2_score", "cv_r2_score", "rmse"]

    zone_df = best_df[best_df["zone"] == zone].copy()

    if len(zone_df) < 2:
        print(f"Not enough models for zone {zone} to create comparison visualization")
        return

    # Prepare data for parallel coordinates plot
    plot_df = zone_df[["feature_set", "best_model"] + metrics].copy()

    # Create a class column combining feature set and model
    plot_df["Model"] = plot_df["feature_set"] + " + " + plot_df["best_model"]

    # For RMSE, lower is better, so we'll negate it for visualization
    if "rmse" in metrics:
        max_rmse = plot_df["rmse"].max()
        plot_df["neg_rmse"] = -1 * plot_df["rmse"] + max_rmse
        metrics = [m if m != "rmse" else "neg_rmse" for m in metrics]

    # Create the parallel coordinates plot
    plt.figure(figsize=(14, 8))
    parallel_coordinates(plot_df, "Model", cols=metrics, colormap="viridis")

    title = f"Model Comparison for Zone {zone} Across Multiple Metrics"
    if "neg_rmse" in metrics:
        plt.text(
            0.5,
            -0.1,
            "Note: RMSE is inverted so higher is better for all metrics",
            ha="center",
            transform=plt.gca().transAxes,
            fontsize=10,
            fontstyle="italic",
        )

    plt.title(title, fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Return the plot for display
    return plt


def run_analysis_combinations_rmse(df, zone_prefixes, trim_outliers_flag=False):
    """
    Run analysis for all the specified feature combinations
    """
    # Define the feature combinations to test
    feature_combinations = [
        {
            "name": "acreage_only",
            "description": "Filter 2015, use acreage",
            "min_year": 2015,
        },
        {
            "name": "acreage_latlon",
            "description": "Filter 2015, use acreage, latitude/longitude",
            "min_year": 2015,
        },
        {
            "name": "acreage_zipcode",
            "description": "Filter 2015, use acreage, zip code encoded",
            "min_year": 2015,
        },
        {
            "name": "acreage_age_latlon",
            "description": "Full data set, acreage, use parcel age, latitude/longitude",
            "min_year": 0,  # Use all data
        },
        {
            "name": "acreage_age_zipcode",
            "description": "Full data set, acreage, parcel age, zip code encoded",
            "min_year": 0,  # Use all data
        },
    ]

    all_results = []

    # Run analysis for each feature combination
    for combo in feature_combinations:
        print(f"\n{'=' * 80}")
        print(f"ANALYZING: {combo['description']}")
        print(f"{'=' * 80}")

        # Run the analysis
        results = compare_regression_models(
            df,
            zone_prefixes,
            feature_set=combo["name"],
            min_year=combo["min_year"],
            trim_outliers_flag=trim_outliers_flag,
        )

        # Summarize the results
        summary_df = summarize_results(results, combo["name"])

        # Add to overall results
        all_results.append(summary_df)

        # Save individual results
        outlier_suffix = "_trimmed" if trim_outliers_flag else ""
        # summary_df.to_csv(
        #     f"regression_results_{combo['name']}{outlier_suffix}.csv", index=False
        # )

    # Combine all results for comparison
    if all_results:
        combined_df = pd.concat(all_results)
        # combined_df.to_csv(
        #     f"combined_regression_results{outlier_suffix}.csv", index=False
        # )

        # Create a summary of best models for each zone and feature set
        best_models = []
        for zone in combined_df["zone"].unique():
            for feature_set in combined_df["feature_set"].unique():
                zone_feature_df = combined_df[
                    (combined_df["zone"] == zone)
                    & (combined_df["feature_set"] == feature_set)
                ]

                if not zone_feature_df.empty and not all(
                    pd.isna(zone_feature_df["r2_score"])
                ):
                    best_idx = zone_feature_df["r2_score"].idxmax()
                    if pd.notna(best_idx):
                        best_model = zone_feature_df.loc[best_idx]
                        best_models.append(
                            {
                                "zone": zone,
                                "feature_set": feature_set,
                                "best_model": best_model["model"],
                                "r2_score": best_model["r2_score"],
                                "cv_r2_score": best_model["cv_r2_score"],
                                "rmse": best_model["rmse"],
                                "samples": best_model["samples"],
                            }
                        )

        best_df = pd.DataFrame(best_models)
        print(best_df)
        # best_df.to_csv(
        #     f"best_models_by_zone_and_features{outlier_suffix}.csv", index=False
        # )

        # Use the multi-metric approach for selecting the best models
        print("\n=== SINGLE METRIC MODEL SELECTION ===")
        print("\n1. Using only R² Score:")
        for zone in best_df["zone"].unique():
            zone_best_df = best_df[best_df["zone"] == zone]
            if not zone_best_df.empty:
                overall_best_idx = zone_best_df["r2_score"].idxmax()
                overall_best = zone_best_df.loc[overall_best_idx]
                print(f"\nZONE: {zone}")
                print(f"Best feature set: {overall_best['feature_set']}")
                print(f"Best model: {overall_best['best_model']}")
                print(f"R² Score: {overall_best['r2_score']:.4f}")
                print(f"CV R² Score: {overall_best['cv_r2_score']:.4f}")
                print(f"RMSE: {overall_best['rmse']:.2f}")

        print("\n2. Using only RMSE:")
        for zone in best_df["zone"].unique():
            zone_best_df = best_df[best_df["zone"] == zone]
            if not zone_best_df.empty:
                overall_best_idx = zone_best_df["rmse"].idxmin()
                overall_best = zone_best_df.loc[overall_best_idx]
                print(f"\nZONE: {zone}")
                print(f"Best feature set: {overall_best['feature_set']}")
                print(f"Best model: {overall_best['best_model']}")
                print(f"R² Score: {overall_best['r2_score']:.4f}")
                print(f"CV R² Score: {overall_best['cv_r2_score']:.4f}")
                print(f"RMSE: {overall_best['rmse']:.2f}")

        print("\n=== MULTI-METRIC MODEL SELECTION ===")
        # Call the multi-metric selection function with different weight configurations
        print("\nBalanced weighting (33% each):")
        best_models_balanced = find_best_models_multi_metric(
            best_df, weight_r2=0.33, weight_cv_r2=0.34, weight_rmse=0.33
        )

        print("\nCross-validation emphasis (50% CV R²):")
        best_models_cv = find_best_models_multi_metric(
            best_df, weight_r2=0.25, weight_cv_r2=0.50, weight_rmse=0.25
        )

        print("\nRMSE emphasis (50% RMSE):")
        best_models_rmse = find_best_models_multi_metric(
            best_df, weight_r2=0.25, weight_cv_r2=0.25, weight_rmse=0.50
        )

        # Create comparison plot of best R² scores by feature set for each zone
        for zone in best_df["zone"].unique():
            zone_best_df = best_df[best_df["zone"] == zone]
            if (
                len(zone_best_df) >= 2
            ):  # Only create plot if we have at least 2 feature sets
                plt.figure(figsize=(12, 6))

                # Sort by R² score
                zone_best_df = zone_best_df.sort_values("r2_score", ascending=False)

                # Create bar chart
                bars = plt.bar(
                    zone_best_df["feature_set"], zone_best_df["r2_score"], alpha=0.7
                )

                # Add model name to each bar
                for i, bar in enumerate(bars):
                    model_name = zone_best_df.iloc[i]["best_model"]
                    plt.text(
                        bar.get_x() + bar.get_width() / 2,
                        0.02,
                        model_name,
                        ha="center",
                        rotation=90,
                        color="white",
                        fontweight="bold",
                    )

                plt.axhline(
                    y=0.7,
                    color="r",
                    linestyle="--",
                    label="R² = 0.7 (Good fit threshold)",
                )
                plt.title(
                    f"Feature Set Comparison for Zone: {zone}{outlier_suffix}",
                    fontsize=14,
                )
                plt.xlabel("Feature Set")
                plt.ylabel("Best R² Score")
                plt.xticks(rotation=45, ha="right")
                plt.ylim(0, 1.0)
                plt.legend()
                plt.tight_layout()

                # plt.savefig(
                #     f"feature_set_comparison_zone_{zone}{outlier_suffix}.png",
                #     dpi=300,
                #     bbox_inches="tight",
                # )
                plt.close()

    return combined_df if all_results else None


def run_analysis_combinations(df, zone_prefixes, trim_outliers_flag=False):
    """
    Run analysis for all the specified feature combinations
    """
    # Define the feature combinations to test
    feature_combinations = [
        {
            "name": "acreage_only",
            "description": "Filter 2015, use acreage",
            "min_year": 2015,
        },
        {
            "name": "acreage_latlon",
            "description": "Filter 2015, use acreage, latitude/longitude",
            "min_year": 2015,
        },
        {
            "name": "acreage_zipcode",
            "description": "Filter 2015, use acreage, zip code encoded",
            "min_year": 2015,
        },
        {
            "name": "acreage_age_latlon",
            "description": "Full data set, acreage, use parcel age, latitude/longitude",
            "min_year": 0,  # Use all data
        },
        {
            "name": "acreage_age_zipcode",
            "description": "Full data set, acreage, parcel age, zip code encoded",
            "min_year": 0,  # Use all data
        },
    ]

    all_results = []

    # Run analysis for each feature combination
    for combo in feature_combinations:
        print(f"\n{'=' * 80}")
        print(f"ANALYZING: {combo['description']}")
        print(f"{'=' * 80}")

        # Run the analysis
        results = compare_regression_models(
            df,
            zone_prefixes,
            feature_set=combo["name"],
            min_year=combo["min_year"],
            trim_outliers_flag=trim_outliers_flag,
        )

        # Summarize the results
        summary_df = summarize_results(results, combo["name"])

        # Add to overall results
        all_results.append(summary_df)

        # Save individual results
        outlier_suffix = "_trimmed" if trim_outliers_flag else ""
        # summary_df.to_csv(
        #     f"regression_results_{combo['name']}{outlier_suffix}.csv", index=False
        # )

    # Combine all results for comparison
    if all_results:
        combined_df = pd.concat(all_results)
        # combined_df.to_csv(
        #     f"combined_regression_results{outlier_suffix}.csv", index=False
        # )

        # Create a summary of best models for each zone and feature set
        best_models = []
        for zone in combined_df["zone"].unique():
            for feature_set in combined_df["feature_set"].unique():
                zone_feature_df = combined_df[
                    (combined_df["zone"] == zone)
                    & (combined_df["feature_set"] == feature_set)
                ]

                if not zone_feature_df.empty and not all(
                    pd.isna(zone_feature_df["r2_score"])
                ):
                    best_idx = zone_feature_df["r2_score"].idxmax()
                    if pd.notna(best_idx):
                        best_model = zone_feature_df.loc[best_idx]
                        best_models.append(
                            {
                                "zone": zone,
                                "feature_set": feature_set,
                                "best_model": best_model["model"],
                                "r2_score": best_model["r2_score"],
                                "cv_r2_score": best_model["cv_r2_score"],
                                "rmse": best_model["rmse"],
                                "samples": best_model["samples"],
                            }
                        )

        best_df = pd.DataFrame(best_models)
        print(best_df)
        # best_df.to_csv(
        #     f"best_models_by_zone_and_features{outlier_suffix}.csv", index=False
        # )

        # For each zone, find the overall best feature set/model combination
        print("\n--- OVERALL BEST MODEL & FEATURE SET FOR EACH ZONE ---")
        for zone in best_df["zone"].unique():
            zone_best_df = best_df[best_df["zone"] == zone]
            if not zone_best_df.empty:
                overall_best_idx = zone_best_df["r2_score"].idxmax()
                overall_best = zone_best_df.loc[overall_best_idx]
                print(f"\nZONE: {zone}")
                print(f"Best feature set: {overall_best['feature_set']}")
                print(f"Best model: {overall_best['best_model']}")
                print(f"R² Score: {overall_best['r2_score']:.4f}")
                print(f"CV R² Score: {overall_best['cv_r2_score']:.4f}")
                print(f"RMSE: {overall_best['rmse']:.2f}")

        # Create comparison plot of best R² scores by feature set for each zone
        for zone in best_df["zone"].unique():
            zone_best_df = best_df[best_df["zone"] == zone]
            if (
                len(zone_best_df) >= 2
            ):  # Only create plot if we have at least 2 feature sets
                plt.figure(figsize=(12, 6))

                # Sort by R² score
                zone_best_df = zone_best_df.sort_values("r2_score", ascending=False)

                # Create bar chart
                bars = plt.bar(
                    zone_best_df["feature_set"], zone_best_df["r2_score"], alpha=0.7
                )

                # Add model name to each bar
                for i, bar in enumerate(bars):
                    model_name = zone_best_df.iloc[i]["best_model"]
                    plt.text(
                        bar.get_x() + bar.get_width() / 2,
                        0.02,
                        model_name,
                        ha="center",
                        rotation=90,
                        color="white",
                        fontweight="bold",
                    )

                plt.axhline(
                    y=0.7,
                    color="r",
                    linestyle="--",
                    label="R² = 0.7 (Good fit threshold)",
                )
                plt.title(
                    f"Feature Set Comparison for Zone: {zone}{outlier_suffix}",
                    fontsize=14,
                )
                plt.xlabel("Feature Set")
                plt.ylabel("Best R² Score")
                plt.xticks(rotation=45, ha="right")
                plt.ylim(0, 1.0)
                plt.legend()
                plt.tight_layout()

                # plt.savefig(
                #     f"feature_set_comparison_zone_{zone}{outlier_suffix}.png",
                #     dpi=300,
                #     bbox_inches="tight",
                # )
                plt.close()

    return combined_df if all_results else None


In [ ]:
# Main execution
def main(df, zone_prefixes):
    # Run analysis with default settings (no outlier trimming)
    print("\nRunning analysis without outlier trimming...")
    combined_results = run_analysis_combinations_rmse(
        df, zone_prefixes, trim_outliers_flag=False
    )

    # Run analysis with outlier trimming
    print("\nRunning analysis with outlier trimming...")
    combined_results_trimmed = run_analysis_combinations_rmse(
        df, zone_prefixes, trim_outliers_flag=True
    )

    print(
        "\nAnalysis complete! Check the CSV files and PNG visualizations for detailed results."
    )

    return combined_results, combined_results_trimmed


# To execute this script:
combined_results, combined_results_trimmed = main(result, ["R3", "R4", "R5"])

In [7]:
result["tax_per_acre_parcel"] = result["Property_Tax_Value"] / result["acreage"]

In [ ]:
r1_data = result[result["assigned_zoning_bucket"] == "R1"]
threshold = r1_data["tax_per_acre_parcel"].quantile(0.99)
trimmed = r1_data[r1_data["tax_per_acre_parcel"] <= threshold]

plt.figure(figsize=(10, 6))
plt.hist(trimmed["tax_per_acre_parcel"], bins=50, color="skyblue", edgecolor="black")
plt.title("Trimmed Histogram of Tax per Acre (Below 99th Percentile) for R1 Parcels")
plt.xlabel("Tax per Acre")
plt.ylabel("Number of Parcels")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()


In [22]:
# Map tax_per_acre and tax_per_acre_recent to result
result["tax_per_acre_zone"] = result["assigned_zoning_bucket"].map(
    lambda z: zone_summary.get(z, {}).get("tax_per_acre", None)
)
result["tax_per_acre_zone_recent"] = result["assigned_zoning_bucket"].map(
    lambda z: zone_summary.get(z, {}).get("tax_per_acre_recent", None)
)


In [ ]:
result["Property_Tax_Value"].sum() * 0.2478

In [ ]:
result["acreage"].sum()

In [236]:
result.to_file("final_parcels_merged.geojson", driver="GeoJSON")

In [ ]:
result.to_file("final_parcels_merged.shp", driver="ESRI Shapefile")


## SB 79 Parcel Analysis
Import SB 79 relevant transit stops geospatial data and use efficiency metrics and regression models to approximate tax increase for affected parcels

In [25]:
# Load the station shapefile
stations = gpd.read_file("data/Metro_Stops_SB_79.geojson")
stations = stations[["79_Tier", "geometry"]]

# Ensure the GeoDataFrame has a projected CRS in feet/meters for buffering (1 mile ≈ 1609.34 meters)
if stations.crs is None or stations.crs.is_geographic:
    # Reproject to a projected CRS suitable for LA (e.g., EPSG:6420 or 2229)
    stations = stations.to_crs(
        epsg=6420
    )  # NAD83 / California zone 5 (ftUS) or 2229 depending on your data

tier_1 = stations[stations["79_Tier"] == 1]
tier_2 = stations[stations["79_Tier"] == 2]


#### Create catchment buffers to determine parcels affected by SB 79 

In [ ]:
# Create 1-mile buffers around each station (1 mile ≈ 5280 feet)
catchment_buffers_1 = tier_1.copy()
catchment_buffers_1["geometry"] = tier_1.buffer(2640)  # 1/2 mile buffer in feet

catchment_buffers_2 = tier_2.copy()
catchment_buffers_2["geometry"] = tier_2.buffer(2640)  # 1/2 mile buffer in feet

catchment_buffers_1_q = tier_1.copy()
catchment_buffers_1_q["geometry"] = tier_1.buffer(1320)  # 1/2 mile buffer in feet

catchment_buffers_2_q = tier_2.copy()
catchment_buffers_2_q["geometry"] = tier_2.buffer(1320)  # 1/2 mile buffer in feet


# Optional: merge all buffers into a single multipolygon for filtering
catchment_union_1_h = catchment_buffers_1.unary_union
catchment_union_2_h = catchment_buffers_2.unary_union
catchment_union_1_q = catchment_buffers_1_q.unary_union
catchment_union_2_q = catchment_buffers_2_q.unary_union


catchment_gdf_1_h = gpd.GeoDataFrame(
    geometry=[catchment_union_1_h], crs=catchment_buffers_1.crs
)

# Ensure other_gdf is in the same CRS
other_gdf = result.to_crs(catchment_gdf_1_h.crs)
# Ensure both are in the same CRS (should be, if projected correctly earlier)
assert other_gdf.crs == catchment_gdf_1_h.crs


In [ ]:
union_1_qtr = catchment_union_1_q
union_1_half = catchment_union_1_h
union_2_qtr = catchment_union_2_q
union_2_half = catchment_union_2_h


def determine_upzoned_zone(geom):
    if geom.intersects(union_1_qtr):
        return "R5"
    elif geom.intersects(union_1_half):
        return "R4"
    elif geom.intersects(union_2_qtr):
        return "R4"
    elif geom.intersects(union_2_half):
        return "R3"
    else:
        return None


# other_gdf["upzoned"] = other_gdf.geometry.apply(determine_upzoned_zone)

tqdm.pandas()  # This monkey-patches pandas to support progress_apply

# Use progress_apply instead of apply
other_gdf["upzoned"] = other_gdf.geometry.progress_apply(determine_upzoned_zone)


In [104]:
copy_gdf = other_gdf.copy()

In [128]:
other_gdf = copy_gdf

In [ ]:
other_gdf["upzoned"].value_counts()

In [131]:
# Define zoning rank (higher = more intense)
zoning_rank = {"Parking": 0, "R1": 1, "R2": 2, "R3": 3, "R4": 4, "R5": 5}

# Assign ranks for comparison
other_gdf["assigned_rank"] = other_gdf["assigned_zoning_bucket"].map(zoning_rank)
other_gdf["upzoned_rank"] = other_gdf["upzoned"].map(zoning_rank)

# Null out any proposed upzoning that would downzone or do nothing
# (optional: if you want to allow same-zone upzones, use < instead of <=)
other_gdf.loc[other_gdf["upzoned_rank"] <= other_gdf["assigned_rank"], "upzoned"] = None

# Drop the temporary rank columns if you want
other_gdf.drop(columns=["assigned_rank", "upzoned_rank"], inplace=True)


In [ ]:
other_gdf["upzoned"].value_counts()

#### Calculate tax increase using tax/acre metrics

In [133]:
# 1. Total acres by original zoning bucket
acres_by_original_zone = (
    other_gdf.groupby("assigned_zoning_bucket")["acreage"].sum().reset_index()
)
acres_by_original_zone.columns = ["zoning_bucket", "total_acres"]

# 2. Create a fallback zoning bucket using 'upzoned' if it exists, else 'assigned_zoning_bucket'
other_gdf["zoning_bucket_with_upzoning"] = other_gdf["upzoned"].fillna(
    other_gdf["assigned_zoning_bucket"]
)

# 3. Total acres by fallback zoning bucket (upzoned if available)
acres_by_upzoned_zone = (
    other_gdf.groupby("zoning_bucket_with_upzoning")["acreage"].sum().reset_index()
)
acres_by_upzoned_zone.columns = ["zoning_bucket", "total_acres_with_upzoning"]

# 4. Merge for comparison (optional)
comparison = pd.merge(
    acres_by_original_zone, acres_by_upzoned_zone, on="zoning_bucket", how="outer"
).fillna(0)


In [134]:
comparison["diff"] = comparison["total_acres_with_upzoning"] - comparison["total_acres"]

In [ ]:
# Only include zones with net acreage gain
comparison_positive = comparison[comparison["diff"] > 0].copy()

# Add tax_per_acre_recent from zone_summary
comparison_positive["tax_per_acre_recent"] = comparison_positive["zoning_bucket"].map(
    lambda z: zone_summary.get(z, {}).get("tax_per_acre_recent", 0)
)

# Calculate estimated new tax from upzoning
comparison_positive["estimated_tax_increase"] = (
    comparison_positive["diff"] * comparison_positive["tax_per_acre_recent"]
)

# Sum total tax increase
total_tax_increase = comparison_positive["estimated_tax_increase"].sum() * 0.2478

print(
    f"💰 Total estimated tax increase from net upzoning acreage: ${total_tax_increase:,.0f}"
)


In [ ]:
comparison

In [ ]:
comparison["total_acres_with_upzoning"].sum()

In [ ]:
comparison["total_acres"].sum()

In [ ]:
# STEP 4: Estimate new tax and difference
def estimate_new_tax(row):
    new_zone = row["upzoned"]
    if pd.notnull(new_zone) and new_zone in zone_summary:
        return zone_summary[new_zone]["tax_per_acre_recent"] * row["acreage"]
    return row["Property_Tax_Value"]


other_gdf["estimated_upzoned_tax"] = other_gdf.apply(estimate_new_tax, axis=1)
other_gdf["estimated_tax_increase"] = (
    other_gdf["estimated_upzoned_tax"] - other_gdf["Property_Tax_Value"]
)

# STEP 5: Optional — Summarize total gain
tax_gain_summary = (
    other_gdf[other_gdf["upzoned"].notnull()]
    .groupby("upzoned")["estimated_tax_increase"]
    .sum()
    .sort_values(ascending=False)
)

print(tax_gain_summary)

#### Estimate tax increase with best performing regression models

In [ ]:
def build_hardcoded_best_models(df, trim_outliers_flag=False):
    """
    Build models based on the hardcoded best configurations from the original analysis
    """
    print(
        "\nBuilding best prediction models for each zoning bucket based on original analysis..."
    )

    # Hardcoded best model configurations from the original analysis results
    best_model_configs = [
        {
            "zone": "R1",
            "feature_set": "acreage_latlon",
            "model_type": "Random Forest",
            "r2_score": 0.6840,
            "cv_r2_score": 0.1949,
            "rmse": 12566.23,
        },
        {
            "zone": "R2",
            "feature_set": "acreage_latlon",
            "model_type": "Random Forest",
            "r2_score": 0.6390,
            "cv_r2_score": -0.0920,
            "rmse": 5336.64,
        },
        {
            "zone": "R3",
            "feature_set": "acreage_age_zipcode",
            "model_type": "Random Forest",
            "r2_score": 0.6643,
            "cv_r2_score": 0.2030,
            "rmse": 14791.45,
        },
        {
            "zone": "R4",
            "feature_set": "acreage_age_latlon",
            "model_type": "Random Forest",
            "r2_score": 0.5918,
            "cv_r2_score": 0.3867,
            "rmse": 40605.66,
        },
        # {
        #     "zone": "R5",
        #     "feature_set": "acreage_age_latlon",
        #     "model_type": "Random Forest",
        #     "r2_score": 0.2033,
        #     "cv_r2_score": -0.0849,
        #     "rmse": 18870.68,
        # },
        {
            "zone": "R5",
            "feature_set": "acreage_age_latlon",
            "model_type": "Random Forest",
            "r2_score": 0.7350,
            "cv_r2_score": -1.1099,
            "rmse": 1123487.58,
        },
        {
            "zone": "Parking",
            "feature_set": "acreage_age_latlon",
            "model_type": "Linear Regression",
            "r2_score": 0.3204,
            "cv_r2_score": 0.2190,
            "rmse": 11800.93,
        },
    ]

    # Now train models using these hardcoded configurations
    best_zone_models = {}

    for config in best_model_configs:
        zone = config["zone"]
        model_type = config["model_type"]
        feature_set = config["feature_set"]
        r2_score_val = config["r2_score"]
        cv_r2_score_val = config["cv_r2_score"]
        rmse_val = config["rmse"]

        try:
            # Print information
            print(f"\nZONE: {zone}")
            print(f"Best feature set: {feature_set}")
            print(f"Best model: {model_type}")
            print(f"R² Score: {r2_score_val:.4f}")
            print(f"CV R² Score: {cv_r2_score_val:.4f}")
            print(f"RMSE: {rmse_val:.2f}")

            # Determine features based on the best feature set
            use_age = "_age" in feature_set
            min_year = 0 if use_age else 2015

            # Prepare data for this zone with appropriate features
            df_processed = prepare_data(
                df,
                min_year=min_year,
                trim_outliers_flag=trim_outliers_flag,
                use_age=use_age,
            )

            # Filter for just this zone
            zone_data = df_processed[
                df_processed["assigned_zoning_bucket"] == zone
            ].copy()

            if len(zone_data) < 10:
                print(
                    f"Skipping zone {zone} - insufficient data (only {len(zone_data)} samples)"
                )
                continue

            # Set up features based on the best feature set
            base_features = ["acreage"]
            if use_age:
                base_features.append("Parcel_age")
            if "latlon" in feature_set:
                base_features.extend(["Location Latitude", "Location Longitude"])

            # Add zip code features if needed
            feature_columns = base_features.copy()
            if "zipcode" in feature_set:
                # Add the one-hot encoding for zip codes
                if "zip_code" in zone_data.columns:
                    zip_dummies = pd.get_dummies(zone_data["zip_code"], prefix="zip")
                    zone_data = pd.concat([zone_data, zip_dummies], axis=1)
                    zip_columns = [
                        col for col in zone_data.columns if col.startswith("zip_")
                    ]
                    feature_columns.extend(zip_columns)

            # Prepare X and y
            X = zone_data[feature_columns].values
            y = zone_data["Property_Tax_Value"].values

            # Instantiate the best model type
            if model_type == "Linear Regression":
                model = LinearRegression()
            elif model_type == "Ridge Regression":
                model = Ridge(alpha=1.0)
            elif model_type == "Lasso Regression":
                model = Lasso(alpha=0.1)
            elif model_type == "ElasticNet":
                model = ElasticNet(alpha=0.1, l1_ratio=0.5)
            elif model_type == "SVR":
                model = Pipeline(
                    [("scaler", StandardScaler()), ("svr", LinearSVR(max_iter=10000))]
                )
            elif model_type == "Random Forest":
                model = RandomForestRegressor(
                    n_estimators=50, random_state=42, n_jobs=4
                )
            elif model_type == "Gradient Boosting":
                model = GradientBoostingRegressor(n_estimators=50, random_state=42)
            else:
                print(f"Unknown model type {model_type} for zone {zone}")
                continue

            # Train the model on the full dataset for this zone
            model.fit(X, y)

            # Store all zip_columns for this zone's model
            all_zip_columns = []
            if "zipcode" in feature_set:
                all_zip_columns = [
                    col for col in zone_data.columns if col.startswith("zip_")
                ]

            # Store the trained model with its configuration
            best_zone_models[zone] = {
                "model": model,
                "feature_set": feature_set,
                "features": feature_columns,
                "use_age": use_age,
                "uses_zipcode": "zipcode" in feature_set,
                "zip_columns": all_zip_columns,  # Store all possible zip columns
                "r2_score": r2_score_val,
                "cv_r2_score": cv_r2_score_val,
                "rmse": rmse_val,
            }
            print(f"Successfully trained model for zone {zone}")

        except Exception as e:
            print(f"Error processing zone {zone}: {str(e)}")

    print(best_zone_models)
    return best_zone_models


# def predict_property_tax_with_models(row, best_zone_models, new_building=True):
#     """
#     Predict property tax using the best model for the target zoning bucket.
#     If new_building is True, age will be set to 0 for models using age.
#     Only applies if 'upzoned' is a real zone different from the original.
#     """
#     target_zone = row["upzoned"]

#     # Skip prediction if not an actual upzone or zone model doesn't exist
#     if pd.isnull(target_zone) or target_zone not in best_zone_models:
#         return row["Property_Tax_Value"]

#     # Get model config for the upzoned zone
#     model_config = best_zone_models[target_zone]
#     model = model_config["model"]
#     feature_set = model_config["feature_set"]
#     use_age = model_config["use_age"]
#     uses_zipcode = model_config["uses_zipcode"]
#     zip_columns = model_config.get("zip_columns", [])

#     # Construct feature vector
#     feature_values = [row["acreage"]]

#     if use_age:
#         age = 0 if new_building else max(0, 2025 - row.get("Effective Year", 2025))
#         feature_values.append(age)

#     if "latlon" in feature_set:
#         feature_values.append(row["Location Latitude"])
#         feature_values.append(row["Location Longitude"])

#     if uses_zipcode:
#         zip_code = row.get("zip_code")
#         for col in zip_columns:
#             col_zip = col.split("_")[1]
#             feature_values.append(1 if str(zip_code) == col_zip else 0)

#     try:
#         X_predict = np.array(feature_values).reshape(1, -1)
#         if X_predict.shape[1] != len(model_config["features"]):
#             print(f"Feature mismatch for zone {target_zone}")
#             return row["Property_Tax_Value"]
#         predicted_tax = model.predict(X_predict)[0]
#         return max(0, predicted_tax)
#     except Exception as e:
#         print(f"Prediction error for zone {target_zone}: {str(e)}")
#         return row["Property_Tax_Value"]


def predict_property_tax_with_models(row, best_zone_models, new_building=True):
    """
    Predict property tax using the best model for the target zoning bucket
    If new_building is True, age will be set to 0 for models using age
    """
    target_zone = row["upzoned"]

    # If no upzoning or the target zone doesn't have a model, return current tax value
    if pd.isnull(target_zone) or target_zone not in best_zone_models:
        return row["Property_Tax_Value"]

    # Get the best model and its configuration for the target zone
    model_config = best_zone_models[target_zone]
    model = model_config["model"]
    feature_set = model_config["feature_set"]
    use_age = model_config["use_age"]
    uses_zipcode = model_config["uses_zipcode"]
    zip_columns = model_config.get("zip_columns", [])

    # Build feature vector in the correct order
    feature_values = []

    # Always add acreage
    feature_values.append(row["acreage"])

    # Add age if used
    if use_age:
        if new_building:
            feature_values.append(0)  # New building has age = 0
        else:
            current_year = 2025
            effective_year = (
                row["Effective Year"]
                if pd.notna(row["Effective Year"])
                else current_year
            )
            feature_values.append(current_year - effective_year)

    # Add lat/lon if used
    if "latlon" in feature_set:
        feature_values.append(row["Location Latitude"])
        feature_values.append(row["Location Longitude"])

    # Handle zip code features if needed
    if uses_zipcode:
        # Get the zip code for this row
        zip_code = (
            row["zip_code"] if "zip_code" in row and pd.notna(row["zip_code"]) else None
        )

        # Create one-hot encoding for this zip code
        for col in zip_columns:
            # Extract the zip from the column name (format: "zip_XXXXX")
            col_zip = col.split("_")[1]
            feature_values.append(
                1 if zip_code is not None and col_zip == str(zip_code) else 0
            )

    # Make prediction
    try:
        # Reshape for single prediction
        X_predict = np.array(feature_values).reshape(1, -1)

        # Check if feature count matches model expectation
        expected_features = len(model_config["features"])
        if X_predict.shape[1] != expected_features:
            print(
                f"Feature mismatch for zone {target_zone}: expected {expected_features}, got {X_predict.shape[1]}"
            )
            return row["Property_Tax_Value"]

        # Predict tax value
        predicted_tax = model.predict(X_predict)[0]

        # Ensure predicted value is not negative
        return max(0, predicted_tax)

    except Exception as e:
        print(f"Error predicting tax for zone {target_zone}: {str(e)}")
        # Fall back to current tax value if prediction fails
        return row["Property_Tax_Value"]


def estimate_new_tax_with_models(df, best_zone_models, new_building=True):
    """
    Estimate new tax values for upzoned parcels using the best ML models
    """
    # Apply the prediction function to each row
    tqdm.pandas(desc="Estimating new tax values")
    df["estimated_upzoned_tax_model"] = df.progress_apply(
        lambda row: predict_property_tax_with_models(
            row, best_zone_models, new_building
        ),
        axis=1,
    )

    # Calculate tax increase
    df["estimated_tax_increase_model"] = (
        df["estimated_upzoned_tax_model"] - df["Property_Tax_Value"]
    )

    # Also run the simple per-acre method for comparison (assuming this function exists)
    if "estimate_new_tax" in globals():
        df["estimated_upzoned_tax_per_acre"] = df.progress_apply(
            estimate_new_tax, axis=1
        )
        df["estimated_tax_increase_per_acre"] = (
            df["estimated_upzoned_tax_per_acre"] - df["Property_Tax_Value"]
        )

    return df


def compare_estimation_methods(df):
    """
    Compare the ML model estimation results with the simple per-acre method
    """
    # Only consider rows with upzoning
    upzoned_df = df[df["upzoned"].notnull()].copy()

    if len(upzoned_df) == 0:
        print("No upzoned parcels to compare")
        return

    # Calculate summary statistics
    total_current_tax = upzoned_df["Property_Tax_Value"].sum()
    total_model_tax = upzoned_df["estimated_upzoned_tax_model"].sum()

    model_increase = total_model_tax - total_current_tax

    print("\n=== TAX ESTIMATION COMPARISON ===")
    print(f"Total Current Tax: ${total_current_tax:,.2f}")
    print(f"Total Estimated Tax (ML Models): ${total_model_tax:,.2f}")

    # Only include per-acre method if it was calculated
    if "estimated_upzoned_tax_per_acre" in upzoned_df.columns:
        total_per_acre_tax = upzoned_df["estimated_upzoned_tax_per_acre"].sum()
        per_acre_increase = total_per_acre_tax - total_current_tax
        print(f"Total Estimated Tax (Per Acre): ${total_per_acre_tax:,.2f}")
        print(
            f"Total Tax Increase (Per Acre): ${per_acre_increase:,.2f} ({per_acre_increase / total_current_tax:.1%})"
        )

    print(
        f"Total Tax Increase (ML Models): ${model_increase:,.2f} ({model_increase / total_current_tax:.1%})"
    )

    # Compare by zone
    print("\n=== TAX INCREASE BY ZONE ===")

    # Group by target upzoned zone and calculate total increase for each method
    agg_dict = {"Property_Tax_Value": "sum", "estimated_tax_increase_model": "sum"}

    if "estimated_tax_increase_per_acre" in upzoned_df.columns:
        agg_dict["estimated_tax_increase_per_acre"] = "sum"

    zone_comparison = upzoned_df.groupby("upzoned").agg(agg_dict).reset_index()

    # Calculate percentage increases
    zone_comparison["model_increase_pct"] = (
        zone_comparison["estimated_tax_increase_model"]
        / zone_comparison["Property_Tax_Value"]
        * 100
    )

    if "estimated_tax_increase_per_acre" in zone_comparison.columns:
        zone_comparison["per_acre_increase_pct"] = (
            zone_comparison["estimated_tax_increase_per_acre"]
            / zone_comparison["Property_Tax_Value"]
            * 100
        )

    # Sort by ML model tax increase
    zone_comparison = zone_comparison.sort_values(
        "estimated_tax_increase_model", ascending=False
    )

    # Print results
    for _, row in zone_comparison.iterrows():
        zone = row["upzoned"]
        current = row["Property_Tax_Value"]
        model_inc = row["estimated_tax_increase_model"]
        model_pct = row["model_increase_pct"]

        print(f"\nZone: {zone}")
        print(f" Current Tax: ${current:,.2f}")
        print(f" ML Model Increase: ${model_inc:,.2f} ({model_pct:.1f}%)")

        if "estimated_tax_increase_per_acre" in row:
            acre_inc = row["estimated_tax_increase_per_acre"]
            acre_pct = row["per_acre_increase_pct"]
            print(f" Per Acre Increase: ${acre_inc:,.2f} ({acre_pct:.1f}%)")

    # Create visualization comparing methods
    plt.figure(figsize=(12, 8))

    # Sort by zone name for better readability
    plot_df = zone_comparison.sort_values("upzoned")
    x = range(len(plot_df))
    width = 0.35

    plt.bar(
        [i - width / 2 for i in x],
        plot_df["estimated_tax_increase_model"],
        width=width,
        label="ML Model Estimate",
        color="blue",
        alpha=0.7,
    )

    if "estimated_tax_increase_per_acre" in plot_df.columns:
        plt.bar(
            [i + width / 2 for i in x],
            plot_df["estimated_tax_increase_per_acre"],
            width=width,
            label="Per Acre Estimate",
            color="green",
            alpha=0.7,
        )

    plt.xlabel("Target Zoning Bucket")
    plt.ylabel("Estimated Tax Increase ($)")
    plt.title("Comparison of Tax Increase Estimation Methods by Zone")
    plt.xticks(x, plot_df["upzoned"], rotation=45, ha="right")
    plt.legend()
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout()
    # plt.savefig("tax_increase_method_comparison.png", dpi=300, bbox_inches="tight")
    plt.close()

    # Create percentage comparison chart
    if "per_acre_increase_pct" in plot_df.columns:
        plt.figure(figsize=(12, 8))
        plt.bar(
            [i - width / 2 for i in x],
            plot_df["model_increase_pct"],
            width=width,
            label="ML Model Estimate (%)",
            color="blue",
            alpha=0.7,
        )
        plt.bar(
            [i + width / 2 for i in x],
            plot_df["per_acre_increase_pct"],
            width=width,
            label="Per Acre Estimate (%)",
            color="green",
            alpha=0.7,
        )
        plt.xlabel("Target Zoning Bucket")
        plt.ylabel("Percentage Tax Increase (%)")
        plt.title("Comparison of Percentage Tax Increase by Zone")
        plt.xticks(x, plot_df["upzoned"], rotation=45, ha="right")
        plt.legend()
        plt.grid(axis="y", linestyle="--", alpha=0.7)
        plt.tight_layout()
        # plt.savefig(
        #     "tax_increase_percentage_comparison.png", dpi=300, bbox_inches="tight"
        # )
        plt.close()

    return zone_comparison


# Updated main function using hardcoded best models
def main_with_hardcoded_models(df):
    """
    Main function that uses hardcoded best models from the original analysis
    """
    # Build models using the hardcoded best configurations
    best_zone_models = build_hardcoded_best_models(df, trim_outliers_flag=False)
    print("estimating now")
    # Estimate new taxes using the trained models
    df_with_estimates = estimate_new_tax_with_models(df, best_zone_models)

    # Compare the estimation methods
    zone_comparison = compare_estimation_methods(df_with_estimates)

    print("\nAnalysis complete! Check the output files for detailed results.")

    return df_with_estimates, zone_comparison, best_zone_models


# To execute this updated code:
df_with_estimates, zone_comparison, best_zone_models = main_with_hardcoded_models(
    other_gdf
)